## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [23]:
import os
import pandas as pd
import numpy as n
from typing import Generator

def get_file_paths(root: str) -> Generator[str, None, None]:
    """
    Returns the paths for all the files in the subtree of `root`
    """
    for (dir_path, _, file_names) in os.walk(root):
        yield from [os.path.join(dir_path, file_name) for file_name in file_names]

gn_file_paths = [*get_file_paths("./data/guinea_data")]
lr_file_paths = [*get_file_paths("./data/liberia_data")]
sl_file_paths = [*get_file_paths("./data/sl_data")]

In [29]:
DATE: str = 

def normalize_sl(frame: pd.core.frame.DataFrame):
    
    return frame

sl0 = pd.read_csv(sl_file_paths[0])
sl1 = normalize_sl(sl0)
print(sl0)
print(sl1)
# type(mb)

          date                variable Kailahun  Kenema    Kono  Kambia  \
0   2014-10-31              population   465048  653013  325003  341690   
1   2014-10-31             new_noncase        0      10       0       0   
2   2014-10-31           new_suspected        0       4       0       0   
3   2014-10-31            new_probable        0       0       0       0   
4   2014-10-31           new_confirmed        3       2       0       1   
5   2014-10-31             cum_noncase      307     825      85      22   
6   2014-10-31           cum_suspected       18      92      58       4   
7   2014-10-31            cum_probable       32       0       1       0   
8   2014-10-31           cum_confirmed      554     493      38      39   
9   2014-10-31         death_suspected        4       4       2       6   
10  2014-10-31          death_probable       35       0      19       9   
11  2014-10-31         death_confirmed      226     255      18      11   
12  2014-10-31           